Reproduce data processing steps from previous notebook

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# All application types with less than 500 applications 
application_types_to_replace = ['T17', 'T29', 'T15', 'T25', 'T14', 'T2', 'T12', 'T13', 'T9']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# All classifications less than 100
val_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = val_counts.loc[val_counts < 100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
target = app_dummies['IS_SUCCESSFUL']
app_dummies = app_dummies.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
(X_train, X_test, y_train, y_test) = train_test_split(app_dummies, target, random_state=76)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimisation 1: adjusting model structure

In [3]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=49))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                750       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 921
Trainable params: 921
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)
# Train the model
fit_output = nn1.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 1ms/step - loss: 0.6121 - accuracy: 0.6955 - auc: 0.7339
Epoch 2/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5556 - accuracy: 0.7281 - auc: 0.7823
Epoch 3/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7296 - auc: 0.7858
Epoch 4/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7294 - auc: 0.7875
Epoch 5/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7313 - auc: 0.7887
Epoch 6/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5454 - accuracy: 0.7338 - auc: 0.7902
Epoch 7/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7329 - auc: 0.7907
Epoch 8/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7316 - auc: 0.7912
Epoch 9/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5431 - accur

In [5]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 0s - loss: 0.5578 - accuracy: 0.7286 - auc: 0.7830 - 414ms/epoch - 2ms/step
Loss: 0.5577555298805237, Accuracy: 0.7286297082901001, AUC: 0.7829667925834656


Optimisation 2 : autotuning

In [6]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn2 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    units_param = hp.Int('first_units', min_value=1, max_value=10, step=2)
    nn2.add(tf.keras.layers.Dense(units=units_param, activation=activation, input_dim=49))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
      units_param = hp.Int('units_'+ str(i), min_value=1, max_value=10, step=2)
      nn2.add(tf.keras.layers.Dense(units=units_param, activation=activation))

    # Add output layer
    nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn2

In [7]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 4.9 MB/s 


In [8]:
# Define tuner
import keras_tuner as kt
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [9]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 29s]
val_accuracy: 0.7233819365501404

Best val_accuracy So Far: 0.727580189704895
Total elapsed time: 00h 13m 25s
INFO:tensorflow:Oracle triggered exit


In [10]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values


{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 4,
 'tuner/bracket': 2,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/round': 1,
 'tuner/trial_id': '4a1a9ed48e3972fc5bca17ea126c6264',
 'units_0': 7,
 'units_1': 9,
 'units_2': 7,
 'units_3': 7,
 'units_4': 9}

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2a = tf.keras.models.Sequential()

# First hidden layer
nn2a.add(tf.keras.layers.Dense(units=7, activation="sigmoid", input_dim=49))

# Second hidden layer
nn2a.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))
nn2a.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn2a.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn2a.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))

# Output layer
nn2a.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2a.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 7)                 350       
                                                                 
 dense_28 (Dense)            (None, 9)                 72        
                                                                 
 dense_29 (Dense)            (None, 7)                 70        
                                                                 
 dense_30 (Dense)            (None, 7)                 56        
                                                                 
 dense_31 (Dense)            (None, 9)                 72        
                                                                 
 dense_32 (Dense)            (None, 1)                 10        
                                                                 
Total params: 630
Trainable params: 630
Non-trainable 

In [30]:
# Compile the model
nn2a.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)
# Train the model
fit_output = nn2a.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.6775 - accuracy: 0.5644 - auc: 0.6326
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5823 - accuracy: 0.7292 - auc: 0.7424
Epoch 3/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5756 - accuracy: 0.7305 - auc: 0.7506
Epoch 4/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5736 - accuracy: 0.7306 - auc: 0.7547
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5729 - accuracy: 0.7307 - auc: 0.7559
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5725 - accuracy: 0.7313 - auc: 0.7567
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5722 - accuracy: 0.7322 - auc: 0.7575
Epoch 8/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5719 - accuracy: 0.7314 - auc: 0.7584
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5717 - accur

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn2a.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 0s - loss: 0.5663 - accuracy: 0.7255 - auc: 0.7763 - 443ms/epoch - 2ms/step
Loss: 0.5662625432014465, Accuracy: 0.7254810333251953, AUC: 0.7763407826423645


Optimisation 3 : adjust input data

In [16]:
# Remove columns ORGANIZATION, STATUS
# Change Other Classification to all values less than 500
classifications_to_replace = val_counts.loc[val_counts < 500].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Remove columns ORGANIZATION, STATUS
application_df_2 = application_df.drop(['ORGANIZATION', 'STATUS'], axis=1)



In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_2 = pd.get_dummies(application_df_2)

# Split our preprocessed data into our features and target arrays
target = app_dummies_2['IS_SUCCESSFUL']
app_dummies_2 = app_dummies_2.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
(X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(app_dummies_2, target, random_state=76)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler_2 = scaler.fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler_2.transform(X_train_2)
X_test_scaled_2 = X_scaler_2.transform(X_test_2)

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=7, activation="sigmoid", input_dim=39))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 7)                 280       
                                                                 
 dense_34 (Dense)            (None, 9)                 72        
                                                                 
 dense_35 (Dense)            (None, 7)                 70        
                                                                 
 dense_36 (Dense)            (None, 7)                 56        
                                                                 
 dense_37 (Dense)            (None, 9)                 72        
                                                                 
 dense_38 (Dense)            (None, 1)                 10        
                                                                 
Total params: 560
Trainable params: 560
Non-trainable 

In [33]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)

In [34]:
# Train the model
fit_output = nn3.fit(X_train_scaled_2, y_train_2, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.6770 - accuracy: 0.5788 - auc: 0.6295
Epoch 2/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5844 - accuracy: 0.7254 - auc: 0.7391
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5798 - accuracy: 0.7276 - auc: 0.7461
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5790 - accuracy: 0.7269 - auc: 0.7501
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5782 - accuracy: 0.7278 - auc: 0.7502
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5778 - accuracy: 0.7269 - auc: 0.7521
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5773 - accuracy: 0.7261 - auc: 0.7507
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5768 - accuracy: 0.7279 - auc: 0.7534
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5764 - accur

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn3.evaluate(X_test_scaled_2,y_test_2,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 0s - loss: 0.5811 - accuracy: 0.7230 - auc: 0.7558 - 444ms/epoch - 2ms/step
Loss: 0.5811153650283813, Accuracy: 0.7230320572853088, AUC: 0.7558122277259827


Optimisation 4: increasing model complexity of new model

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=15, activation="sigmoid", input_dim=49))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=15, activation="sigmoid"))
nn4.add(tf.keras.layers.Dense(units=15, activation="sigmoid"))
nn4.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
nn4.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 15)                750       
                                                                 
 dense_46 (Dense)            (None, 15)                240       
                                                                 
 dense_47 (Dense)            (None, 15)                240       
                                                                 
 dense_48 (Dense)            (None, 10)                160       
                                                                 
 dense_49 (Dense)            (None, 10)                110       
                                                                 
 dense_50 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,511
Trainable params: 1,511
Non-traina

In [39]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)
# Train the model
fit_output = nn4.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.6709 - accuracy: 0.5940 - auc: 0.6290
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5782 - accuracy: 0.7294 - auc: 0.7457
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5736 - accuracy: 0.7310 - auc: 0.7547
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5726 - accuracy: 0.7305 - auc: 0.7559
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5723 - accuracy: 0.7312 - auc: 0.7571
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5720 - accuracy: 0.7306 - auc: 0.7584
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5715 - accuracy: 0.7306 - auc: 0.7584
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5715 - accuracy: 0.7306 - auc: 0.7592
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5712 - accur

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 0s - loss: 0.5636 - accuracy: 0.7275 - auc: 0.7756 - 433ms/epoch - 2ms/step
Loss: 0.5636324882507324, Accuracy: 0.7274635434150696, AUC: 0.7756137251853943


In [25]:
# Save models as HDF5
nn1.save('AlphabetSoupCharity_optimization_1.h5')
nn2a.save('AlphabetSoupCharity_optimization_2.h5')
nn3.save('AlphabetSoupCharity_optimization_3.h5')
nn4.save('AlphabetSoupCharity_optimization_4.h5')